# **Predict Translation Word And Char Count**

> Prediction of translated Word/Char Count is used as a Quality and/or Validation Check
- toc: true
- branch: master
- badges: false
- comments: true
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2
- image: images/PredictTranslationWordAndCharCount.png
- categories: [Deep Learning,Time series,Regression,   Python,fastai]
- show_tags: true

# **Predict Translation Word And Char Count**

In [2]:
#hide
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'TODANALYTICS/'
# base_dir = ""

Mounted at /content/gdrive


In [3]:
# # ! pip install fastai
# ! pip install fastai2
# ! pip install nbdev

!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 727kB 8.6MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 194kB 22.8MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 1.0MB 28.0MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 92kB 12.6MB/s 
     |████████████████████████████████| 40kB 7.2MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 61kB 9.1MB/s 
     |████████████████████████████████| 2.6MB 53.4MB/s 


In [4]:
# ! pip list | grep fastai
! pip list | grep fastai2

In [5]:
from fastai.tabular.all import *
from fastbook import *

# from fastai.tabular.all import *
# # from fastai2.tabular.all import *

In [6]:
import seaborn as sns
# import os
# import json
%matplotlib inline

In [7]:
!python --version

Python 3.6.9


In [8]:
# PATH='./'
PATH = Path(base_dir + './'); PATH

Path('/content/gdrive/My Drive/TODANALYTICS')

# Get train/valid data

In [9]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 1000): 
        display(df)

## Ingest all E-contributions

In [10]:
# df_E = pd.read_csv(f'{PATH}/contributions/1955-0807y_Pride_ENG_15-0401-b_E-contributions.csv', sep='~')
# df_E = pd.read_csv(f'{PATH}/contributions/1965-0829_SatansEden_ENG_15-1104-b_E-contributions.csv', sep='~')

all_files = glob.glob(f"{PATH}/contributions/*E-contributions.csv")
li = []
for filename in all_files:
    dft = pd.read_csv(filename, index_col=None, header=0, sep='~')
    li.append(dft)
df_E = pd.concat(li, axis=0, ignore_index=True)

df_E.head()

,m_descriptor,t_lan,t_senc,t_version,s_typ,s_rsen,e_id,e_top,be_id,be_top,c_id,c_created_at,c_kind,c_eis,c_base,a_role,u_name,e_content
0,1965-0418x,ENG,1870,18-0101-E1R,n,1,174684,Z,NaN,NaN,224461,2018-03-29 23:10:24.573038,E,0,NaN,EP,kobest,Let us bow our heads.
1,1965-0418x,ENG,1870,18-0101-E1R,n,2,174685,Z,NaN,NaN,224462,2018-03-29 23:10:24.595501,E,0,NaN,EP,kobest,"Lord, as we gather here this fine Easter morning, see the little buds pressing their way out, the bees flying in and getting their portion, the birds singing like their hearts would burst with joy, because there is an Easter."
2,1965-0418x,ENG,1870,18-0101-E1R,n,3,174686,Z,NaN,NaN,224463,2018-03-29 23:10:24.628362,E,0,NaN,EP,kobest,"We believe that You raised up Jesus from the dead, many years ago, today, and we celebrate this memorial day."
3,1965-0418x,ENG,1870,18-0101-E1R,n,4,174687,Z,NaN,NaN,224464,2018-03-29 23:10:24.650119,E,0,NaN,EP,kobest,"And let there come an Easter among us all, today."
4,1965-0418x,ENG,1870,18-0101-E1R,n,5,174688,Z,NaN,NaN,224465,2018-03-29 23:10:24.670806,E,0,NaN,EP,kobest,"May we, as His servants, understand His Word, that we were in His fellowship then, and that now that we are risen with Him and setting together in Heavenly places."


In [11]:
#remove if not used to debug
df_E = df_E.drop(['e_id','t_senc','s_typ','e_top','be_id','be_top','c_created_at','c_kind','c_eis','c_base','a_role','u_name'], axis=1) #each record now unique
df_E.head()

,m_descriptor,t_lan,t_version,s_rsen,c_id,e_content
0,1965-0418x,ENG,18-0101-E1R,1,224461,Let us bow our heads.
1,1965-0418x,ENG,18-0101-E1R,2,224462,"Lord, as we gather here this fine Easter morning, see the little buds pressing their way out, the bees flying in and getting their portion, the birds singing like their hearts would burst with joy, because there is an Easter."
2,1965-0418x,ENG,18-0101-E1R,3,224463,"We believe that You raised up Jesus from the dead, many years ago, today, and we celebrate this memorial day."
3,1965-0418x,ENG,18-0101-E1R,4,224464,"And let there come an Easter among us all, today."
4,1965-0418x,ENG,18-0101-E1R,5,224465,"May we, as His servants, understand His Word, that we were in His fellowship then, and that now that we are risen with Him and setting together in Heavenly places."


In [12]:
#handle NaNs in e_content
e_content_nans = df_E['e_content'].isna()
df_E[e_content_nans]

,m_descriptor,t_lan,t_version,s_rsen,c_id,e_content
33415,1956-0805,ENG,15-0402-b,1176,454335,NaN
43018,1957-0419,ENG,15-0401-b,505,13306,NaN


In [13]:
#replace e_content NaNs with empty strings
df_E.loc[e_content_nans, 'e_content'] = ''
# df_E.loc[e_content_nans, ['e_content']]
# OR
df_E[df_E['e_content']=='']

,m_descriptor,t_lan,t_version,s_rsen,c_id,e_content
33415,1956-0805,ENG,15-0402-b,1176,454335,
43018,1957-0419,ENG,15-0401-b,505,13306,


In [14]:
#add chars column
df_E['chars'] = [len(e) for e in df_E['e_content']]
# df_E['chars'] = [len(e) if type(e)==str else 1 for e in df_E['e_content']]
df_E.head()

,m_descriptor,t_lan,t_version,s_rsen,c_id,e_content,chars
0,1965-0418x,ENG,18-0101-E1R,1,224461,Let us bow our heads.,21
1,1965-0418x,ENG,18-0101-E1R,2,224462,"Lord, as we gather here this fine Easter morning, see the little buds pressing their way out, the bees flying in and getting their portion, the birds singing like their hearts would burst with joy, because there is an Easter.",225
2,1965-0418x,ENG,18-0101-E1R,3,224463,"We believe that You raised up Jesus from the dead, many years ago, today, and we celebrate this memorial day.",109
3,1965-0418x,ENG,18-0101-E1R,4,224464,"And let there come an Easter among us all, today.",49
4,1965-0418x,ENG,18-0101-E1R,5,224465,"May we, as His servants, understand His Word, that we were in His fellowship then, and that now that we are risen with Him and setting together in Heavenly places.",163


In [15]:
# df_E.loc[e_content_nans, ['e_content','chars']]
# OR
df_E[df_E['chars']==0]

,m_descriptor,t_lan,t_version,s_rsen,c_id,e_content,chars
33415,1956-0805,ENG,15-0402-b,1176,454335,,0
43018,1957-0419,ENG,15-0401-b,505,13306,,0


In [16]:
#add words column
#https://www.geeksforgeeks.org/python-program-to-count-words-in-a-sentence/
df_E['words'] = [len(re.findall(r'\w+', e)) for e in df_E['e_content']]
df_E.head()

,m_descriptor,t_lan,t_version,s_rsen,c_id,e_content,chars,words
0,1965-0418x,ENG,18-0101-E1R,1,224461,Let us bow our heads.,21,5
1,1965-0418x,ENG,18-0101-E1R,2,224462,"Lord, as we gather here this fine Easter morning, see the little buds pressing their way out, the bees flying in and getting their portion, the birds singing like their hearts would burst with joy, because there is an Easter.",225,40
2,1965-0418x,ENG,18-0101-E1R,3,224463,"We believe that You raised up Jesus from the dead, many years ago, today, and we celebrate this memorial day.",109,20
3,1965-0418x,ENG,18-0101-E1R,4,224464,"And let there come an Easter among us all, today.",49,10
4,1965-0418x,ENG,18-0101-E1R,5,224465,"May we, as His servants, understand His Word, that we were in His fellowship then, and that now that we are risen with Him and setting together in Heavenly places.",163,30


In [17]:
#remove BER from t_version
df_E['t_version'] = ['-'.join(e.split('-')[:2]) for e in df_E['t_version']]
df_E.head()

,m_descriptor,t_lan,t_version,s_rsen,c_id,e_content,chars,words
0,1965-0418x,ENG,18-0101,1,224461,Let us bow our heads.,21,5
1,1965-0418x,ENG,18-0101,2,224462,"Lord, as we gather here this fine Easter morning, see the little buds pressing their way out, the bees flying in and getting their portion, the birds singing like their hearts would burst with joy, because there is an Easter.",225,40
2,1965-0418x,ENG,18-0101,3,224463,"We believe that You raised up Jesus from the dead, many years ago, today, and we celebrate this memorial day.",109,20
3,1965-0418x,ENG,18-0101,4,224464,"And let there come an Easter among us all, today.",49,10
4,1965-0418x,ENG,18-0101,5,224465,"May we, as His servants, understand His Word, that we were in His fellowship then, and that now that we are risen with Him and setting together in Heavenly places.",163,30


## Ingest all V-contributions